In [3]:
import os

In [4]:
%pwd

'd:\\FutureExpertData\\Text-Summarizer-Project\\reseach'

In [5]:
os.chdir('../')

In [6]:
%pwd

'd:\\FutureExpertData\\Text-Summarizer-Project'

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path              # Where to store all artifacts related to data ingestion
    source_URL: str             # Where to download the data from (a remote URL)
    local_data_file: Path       # Path to save the downloaded ZIP file
    unzip_dir: Path             # Where to extract the contents of the ZIP


In [ ]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories



In [ ]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories


class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH, 
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root]) #automatically create the folder that will contain the artefacts
        
    # in the corresponding files we defined some constant values, here we access them !
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir= config.root_dir, 
            source_URL= config.source_URL, 
            local_data_file= config.local_data_file,
            unzip_dir= config.unzip_dir
        )
        return data_ingestion_config

In [17]:
import os 
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size 


In [18]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL, 
                filename= self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
           
    
    def extract_zip_file(self):
        """
        Zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """ 
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [20]:
#update the pipeline
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_configuration = DataIngestion(config=data_ingestion_config)
    data_configuration.download_file()
    data_configuration.extract_zip_file()
except Exception as e:
    raise e
    
    

[2025-07-18 09:09:25,237: INFO: common: yaml file: config\config.yaml loaded successfully.]
[2025-07-18 09:09:25,239: INFO: common: yaml file: params.yaml loaded successfully.]
[2025-07-18 09:09:25,240: INFO: common: Created directory at: artifacts]
[2025-07-18 09:09:25,241: INFO: common: Created directory at: artifacts/data_ingestion]
[2025-07-18 09:09:29,875: INFO: 3764822244: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: E6F7:1426E5:131C2EE:15F878E:6879F325
Accept-Ranges: bytes
Date: Fri, 18 Jul 2025 07:09:26 GMT
Via: 1.1 varnish
X-Served-By: cache-ams21082-AMS
X-Cach